**Yapacağınız yorumlar için şimdiden teşekkür ederim. Çalışmayı beğenirseniz oylamayı unutmayın lütfen. Sorulara elimden geldiği kadarıyla cevap vermeye çalışacağım. Çalışmada; sorulara ve geri bildirimlere göre düzeltme ve ekleme yapacağım.**

**Yapacağınız yorumlar, çalışmanın daha iyi ve anlaşılır olmasını sağlayacaktır. **

# <a class="anchor" id="0."></a>**İçindekiler**

* [1. Çalışmanın Özeti](#1.)
* [2. Veri Seti Hakkında Teknik ve Bağlamsa Değerlendirme](#2.)
* [3. Eksik Bilgilerin Doldurulması İçin Yapılan İşlemler](#3.)
* * [3.1. Bazı Sütünların Yeniden Adlandırılması](#3.1.)
* * [3.1. ??? Değerlerinin nan Değeriyle Değiştirilmesi](#3.2.)
* * [3.2. Eksik Bilgiler Dolduruluyor](#3.3.)
* [4. Veri Setinin Grafiklerle İncelenmesi](#4.)
* * [4.1. height, width, ratio ve local Özelliklerinin Önem Sırası](#4.1e1.)
* * [4.2. RadViz Görselleştirmesi](#4.1e2.)
* * [4.3. Paralel Kordinatlar](#4.1e3.)
* * [4.4. Sınıf Dağılımları](#4.1.)
* * [4.5. Yardımcı Fonksiyon:draw_multivarient_plot()](#4.2.)
* * [4.6. Box Plot](#4.3.)
* * [4.7. Violin Plot](#4.4.)
* * [4.8. Point Plot](#4.5.)
* * 4.9. [Özelliklerin Birbirleriyle Olan Korelasyonu](#4.6.)
* * [4.10. Özelliklerin Birbirleriyle Olan İlişkileriinin Pair Plot İle Gösterimi](#4.7.)
* [5. Makine Öğrenmesi Modellerinin Kullanılması](#5.)
* * [5.1. Kullanılacak Makine Öğrenmesi Modelleri](#5.1.)
* * [5.2. Yardımcı Fonksiyonlar:cross_validation_scores-for_various_ml_models()](#5.2.)
* * [5.3. Çapraz Doğrulama Sonuçları](#5.3.)
* * [5.4. Yardımcı Fonksiyon:SVM_GridSearch()](#5.4.)
* * [5.6. En İyi Sonuç Veren SVC Parametrelerinin Elde Edilmesi](#5.5.)
* * [5.7. En İyi SVC Parametrelerine Göre Elde Edilen Sonuçlar](#5.6.)
* * [5.8. ROC ve AUC](#5.7.)
* [6. Denetimsiz Öğrenme](#6.)
* * [6.1. Temel Bileşen Analizi(PCA)](#6.1.)
* * [6.2. PCA Dönüşümünün Gerçekleştirilmesi](#6.2.)
* * [6.3. PCA Değerlerinin Görselleştirilmesi](#6.3.)
* * [6.4. Sadece İki PCA Bileşeniyle Elde Edilen Sonuçlar](#6.4.)
* * [6.5. Optimum PCA Bileşen Sayısının Belirlenmesi](#6.5.)
* * [6.6. Optimum PCA Bileşen Sayısına Göre Sonuçlar](#6.6.)
* [7. Makine Öğrenmesi İş Akışının Otomatikleştirilmesi](#7.)
* [8. Makine Öğrenmesi Modelinin Kaydedilmesi ve Yüklenmesi](#8.)

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="1."></a>**1. Çalışmanın Özeti** 

Bu çalışmada web sayfalarındaki reklam ve reklam olmayan resimlerin sınıflandırılması **meta verilere** göre yapılmıştır. Çalışmanın özeti şöyledir:
1. Veri setinin yüklenmesi
1. Ön işlemlerin yapılması
1. Veri setinin görselleştirilmesi
1. Normalizasyon yapılması
1. Farklı sınıflandırıcılara göre çapraz doğrulama sonuçlarının elde edilmesi
1. Çapraz doğrulaması en iyi sonucun optimizasyonun yapılması
1. PCA veri boyutunun azaltılması
1. PCA ile azaltılan veriler üzerinde sınıflandırma yapılması
1. Pipeline ile makine akışının otomatikleştirilmesi
1. Modelin daha sonra kullanımı için kaydedilmesi ve yeniden yüklenmesi işlemleri



In [ ]:
#Gerekli Kütüphanelerin Yüklenmesi Yapılıyor

import numpy as np # Lineer cebir işlemleri için
import pandas as pd # Veri işleme

#Görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns

#Makine Öğrenmesi
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

#Sistem
import os
import warnings

#Uyarılar devre dışı bıralılıyor
warnings.filterwarnings('ignore')
print(os.listdir("../input"))


In [ ]:
dataset_raw=pd.read_csv("../input/add.csv")
dataset_raw.head(5)

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="2."></a>**2. Veri Seti Hakkında Teknik ve Bağlamsal Değerlendirme**  

* Veri seti 1558 özellik sütunundan oluşmaktadır. 
* 'Unnamed:0' başlıklı sütun veri setindeki örnekleri numaralandırmaktadır. Makine öğrenmesi modeli için bir anlamı olmadığı için çıkartılacaktır.

* '0', '1' ve '2' başlıklı sütunlarda web sayfasındaki resmin yüksekliği, genişliği ve oranı(genişlik/yükseklik) verilmektedir. 
* '3' başlıklı sütun, web sayfasının domaini ile resmin domain'inin  aynı olup olmamasına gösteriyor; 1-->aynı domain, 0--> farklı domain 
* 1558 numaralı sütun, özellik verktörünün ait olduğu sınıfı belirtmektedir. **ad**(reklam resmi) ve **nonad** (reklam olmayan resim) olmak üzere iki sınıf vardır. 
* 4-1557 arasındaki sütunlar, görüntüye ait özellik değerleridir. 4-1557 arasındaki sütunların nasıl elde eldildiğini Kushmerick tarafından yapılan "Learning to Remove Internet Advertisiment" başlıklı bildiride yer almaktadır. Bildirinde yer alan bilgilere göre özellik vektörü 'A' etiketi arasında resimlere ait **caption**(başlık), **alt**(resmi tanıtıcı metin), **url_base**(web sayfasına ait url) ve **url_dest**(tıklandığında gidilecek url)  bilgilerinden türetilmiştir.  Aşağıdaki şekilde 'A' etiketi içerisindeki resim bilgileri yer almaktadır. Tablo resimde yer alan url bilgilerine göre oluşturulmuştur.
[![image](https://i.hizliresim.com/j64vD9.png)](https://hizliresim.com/j64vD9)

Yukarıdaki resimde yer alan etiketlerden türetilen üç örnek aşağıdaki tabloda yer almaktadır.
[![image](https://i.hizliresim.com/Z3LAmA.png)](https://hizliresim.com/Z3LAmA)
Kaynak:http://www.sc.ehu.es/ccwbayes/docencia/mmcc/docs/lecturas-clasificacion/abstracts-resumir/kushmerick99learning.pdf

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="3."></a>**3. Eksik Bilgilerin Doldurulması İçin Yapılan İşlemler** 

Veri setinde eksik bilgi sadece dört özellik sütünunda vardır; height, width, ratio ve local. Önce eksik bilgi içeren sütünları tespit edelim ve sonra eksik bilgileri dolduralım. 

<a class="anchor" id="3.1."></a>**3.1. Bazı Sütünların Yeninden Adlandırılması** <=======> [İçindekiler Menüsüne Git](#0.)

Veri setinde yer alan bazı özellik sütünlarının adları daha anlamlı isimlerle değiştiriliyor. Yeni adlandırmalar yukarıda bahsedilen makalede yer alan bilgilere göre yapılmıştır. 

In [ ]:
#Unnamed: 0 sütununu veri setinden çıkartılır
dataset_raw.drop('Unnamed: 0',inplace=True, axis=1)
#0, 1, 2, 3 ve 1558 başlıklı sütunlar sırasıyla height, width, ratio, local ve target
#olarak değiştirilir
dataset_raw.rename(columns={'0':'height', '1':'width', '2':'ratio(width/height)','3':'local', '1558':'target'}, inplace=True)

dataset_raw.head(5)

<a class="anchor" id="3.2."></a>**3.2.  ??? Değerlerinin nan Değeriyle Değiştirilmesi** <=======> [İçindekiler Menüsüne Git](#0.)

height, width ve ratio sütunlarındaki olmayan bilgiler  '???' işaretiyle doldurulmuştur. '???' işareti, eksik değer olduğunu göstermektedir. Bu sütunlardaki "???" değerlerini nan değerleri ile değiştirelim ve ne kadar eksik değer olduğunu bulalım.


In [ ]:
#??? içeren değerler 'nan' ile doldurulur
dataset=dataset_raw.applymap(lambda x:np.nan if isinstance(x,str) and '?' in x else x)

#eksik bilgi içeren özellikler bulunur

results=dataset.iloc[:,:].isnull().sum()

In [ ]:
null_columns=[]
for index, value in results.iteritems():
    if value !=0:
        print("{} : {}".format(index, value))
        null_columns.append(index)

In [ ]:
#Yukarıdaki hücrede elde edilen sonuçları daha basitçe elde ediliyor.
print(results[results>0])

<a class="anchor" id="3.3."></a>**3.3. Eksik Bilgiler Dolduruluyor** <=======> [İçindekiler Menüsüne Git](#0.)

Eksik bilgiler içeren özellik sütunlarındaki eksik bilgileri dolduralım. Eksik bilgilerin doldurulması şöyle yapılacaktır:
* **height, width** ve **ratio(width/height)** özellikleri, noktalı değerler içerdiği için, ilgili sütunun ortalaması ile doldurulacaktır. 
* **local** özelliği kategorik değer içerdiği için en sık olan değerle doldurulacaktır.

In [ ]:
dataset[null_columns].info()

Öncelikle; **height, width, ratio(width/height) **ve** local** özelliklerinin değişken tipleri sayısal değerlere dönüştürülmelidir

In [ ]:
label_encoder=LabelEncoder()
dataset['target']=label_encoder.fit_transform(dataset['target'])
dataset[null_columns]=dataset[null_columns].astype('float')
dataset[null_columns].info()

In [ ]:
print("0-->{} ve 1-->{}".format(label_encoder.inverse_transform([0])[0],
                                label_encoder.inverse_transform([1])[0]))

In [ ]:
#Tüm eksik satırları göstermek yerine, birkaç eksik satırı görüntüleyelim
#10, 24 ve 72 satırları eksik bilgiler içermektedir.
#Bu satırları görelim.
dataset.iloc[[10,24,72]]

In [ ]:
# height, width ve ratio sütünlarında yer alan eksik bilgiler sütünun ortalaması ile doldurulur
dataset.iloc[:,0:3]=dataset.iloc[:,0:3].fillna(dataset.iloc[:,0:3].dropna().mean())

# local sütünundaki eksik bilgiler en cok olan kategori ile doldurulur
dataset['local']=dataset['local'].fillna(dataset['local'].dropna().mode()[0])

In [ ]:
#Tüm eksik bilgiler doldurulmuştur
#Eksik bilgiler doldurulduktan sonra 10, 24 ve 72 numaralı satırları görelim
dataset.iloc[[10,24,72]]

Veri setindeki 10, 24 ve 72 satırlarnda eksik bilgiler vardı. Bu bilgiler ortalama ile doldurulmuştur. Null değer içeren sütünları kontrol edelim


In [ ]:
dataset[null_columns].info()

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="4."></a>**4. Veri Setinin Grafiklerle İncelenmesl**  

Bu bölümde, veri setinin grafiklerle incelenmesi yapılacaktır.  Yapılacak işlemler şöyledir:
* height, width, ratio ve local özelliklerinin önem dereceleri gösterilecektir
* RadViz ile dört özelliğin hedef değişkene göre dağılımı tekbir grafikte gösterilecektir
* Paralel koordinat grafiğiyle yine, dört özelliğin hedef değişkene göre dağılımı tekbir grafikte gösterilecektir. 
* ad ve nonad olan örnek sayısının gösterilmesi için countplot kullanılacaktır
* height, width, ratio ve local özelliklerinin ad ve nonad sınıfları ile ilişkisi için violin, boxplot ve pointplot kullanılacaktır
* height, width, ratio ve local özelliklerinin bir birleri ile olan ilişkisi için pairplot kullanılacaktır. 

<a class="anchor" id="4.1e1."></a>**4.1. height, width, ratio ve local Özelliklerinin Önem Sırası** <=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
from yellowbrick.features import Rank1D

checked_X=dataset[null_columns]
checked_y=dataset['target']

fig, ax=plt.subplots(figsize=(15,8))
rank1D=Rank1D(features=null_columns, 
              algorithm="shapiro",
             ax=ax)

rank1D.fit(checked_X, checked_y)
rank1D.transform(checked_X)

plt.figure(figsize=(5,8))
rank1D.poof(title="ben")

<a class="anchor" id="4.1e2."></a>**4.2. RadViz Görselleştirmesi** <=======> [İçindekiler Menüsüne Git](#0.)

RadViz, her özellik boyutunu bir dairenin çevresine eşit olarak çizen, ardından dairenin iç tarafındaki noktaları çizen, böylece merkezdeki değerlerini merkezden her bir yaya normalleştiren çok değişkenli bir veri görselleştirme algoritmasıdır. Bu mekanizma, görselleştirmenin boyutsallığını büyük ölçüde genişleterek, bir daireye kolayca sığacak kadar çok boyuta izin verir.

Veri bilimcileri, sınıflar arasındaki ayrımı tespit etmek için bu yöntemi kullanır. Örneğin. özellik setinden öğrenilebilecek bir şey var mı,  yoksa özellik uzayı gürültülü mü, gibi soruların cevabını bulmak için kullanılabilir. 

In [ ]:
from yellowbrick.features import RadViz

fig, ax=plt.subplots(figsize=(15,8))
radViz=RadViz(classes=['ad', 'nonad'], 
              features=null_columns,
             ax=ax
             )
radViz.fit(checked_X, checked_y)
radViz.transform(checked_X)

radViz.poof()

<a class="anchor" id="4.1e3."></a>**4.3. Paralel Kordinatlar**   <=======> [İçindekiler Menüsüne Git](#0.)

Paralel koordinatlar, her bir özellik için dikey eksenin yatay olarak kopyalandığı çok boyutlu özellik görselleştirme tekniğidir. Örnekler, her dikey eksenden, bu özellik için değerlerini temsil eden konuma çizilen tek bir satır segmenti olarak görüntülenir. Bu, birçok boyutun aynı anda görselleştirilmesine izin verir; Aslında sonsuz yatay alan (örneğin bir kaydırma penceresi) verildiğinde, teknik olarak sonsuz sayıda boyut görüntülenebilir!

Veri bilimcileri, benzer sınıflara sahip örneklerin kümelerini tespit etmek ve yüksek varyansa veya farklı dağılımlara sahip özelliklere dikkat çekmek için bu yöntemi kullanır.



In [ ]:
from yellowbrick.features import ParallelCoordinates
fig, ax=plt.subplots(1, figsize=(15,8))
parallelCoordinates=ParallelCoordinates(classes=['ad', 'nonad'],
                                       features=null_columns, 
                                        sample=0.5, 
                                        shuffle=True,
                                        normalize="standard",
                                       ax=ax)

parallelCoordinates.fit(checked_X, checked_y)
parallelCoordinates.transform(checked_X)


parallelCoordinates.poof()

<a class="anchor" id="4.1."></a>**4.4. Sınıf Dağılımları**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
# ad ve nonad sınıflarının dağılımı
print(dataset['target'].value_counts())
sns.countplot(dataset['target'])

<a class="anchor" id="4.2"></a>**4.5. Yardımcı Fonksiyon:draw_multivarient_plot()** <=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
def draw_multivarient_plot(dataset, rows, cols, plot_type):
    column_names=dataset.columns.values
    number_of_column=len(column_names)
    fig, axarr=plt.subplots(rows,cols, figsize=(16,10))

    counter=0
    for i in range(rows):
        for j in range(cols):
             
            if counter>(number_of_column-1):
                break
            if 'violin' in plot_type:
                sns.violinplot(x='target', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'box'in plot_type :
                sns.boxplot(x='target', y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'point' in plot_type:
                sns.pointplot(x='target',y=column_names[counter],data=dataset, ax=axarr[i][j])
            elif 'bar' in plot_type:
                sns.barplot(x='target',y=column_names[counter],data=dataset, ax=axarr[i][j])
                
            counter+=1

Veri setindeki özelliklerin kalite ile ilişkilerini yorumlamak için kullanılabilecek grafik türleri şöyledir: Keman Grafiği (Violin Plot), Kutu Grafiği (Box Plot), Nokta Grafiği (Point Plot) ve Bar Grafiği (Bar Plot)

<a class="anchor" id="4.3."></a>**4.6. Box Plot**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
columns_for_visualization=null_columns + ['target']
print(columns_for_visualization)
draw_multivarient_plot(dataset[columns_for_visualization],rows=2,cols=3,plot_type="box")

<a class="anchor" id="4.4."></a>**4.7. Violin Plot** <=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
draw_multivarient_plot(dataset[columns_for_visualization],rows=2,cols=3,plot_type="violin")

<a class="anchor" id="4.5."></a>**4.8. Point Plot**  <=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
draw_multivarient_plot(dataset[columns_for_visualization],rows=2,cols=3,plot_type="point")

<a class="anchor" id="4.6."></a>**4.9. Özelliklerin Birbirleriyle Olan Korelasyonu**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
# özelliklerin birbirleriyle olan ilişkisini korelasyon matrisi kullanarak gösterebiliriz. 
corr=dataset[columns_for_visualization].corr()
fig, ax = plt.subplots(figsize=(12,12))
sns.heatmap(corr,annot=True,linewidths=.5, ax=ax)

<a class="anchor" id="4.7."></a>**4.10. Özelliklerin Birbiryle Olan İlişkilerinin Pair Plot ile Gösterimi**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
# özelliklerin birbirleriyle olan ilişkisini pairplot kullanarak gösterebiliriz. 
sns.pairplot(dataset[columns_for_visualization], hue="target", markers=["o", "s"])

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="5."></a>**5. Makine Öğrenmesi Modellerinin Kullanılması** 

<a class="anchor" id="5.1."></a>**5.1. Kullanılacak Makine Öğrenmesi Modelleri**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
def get_models():
    models=[]
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(("LR",LogisticRegression()))
    models.append(("NB",GaussianNB()))
    models.append(("KNN",KNeighborsClassifier(n_neighbors = 3)))
    models.append(("DT",DecisionTreeClassifier()))
    models.append(("SVM rbf",SVC()))
    models.append(("SVM linear",SVC(kernel='linear')))
    
    return models

<a class="anchor" id="5.2."></a>**5.2. Yardımcı Fonksiyonlar: cross_validation_scores_for_various_ml_models()**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
def cross_validation_scores_for_various_ml_models(X_cv, y_cv):
    print("Çapraz Doğrulama Başarı Oranları".upper())
    models=get_models()
    results=[]
    names= []

    for name, model in models:
        kfold=KFold(n_splits=5,shuffle=True, random_state=22)
        cv_result=cross_val_score(model,X_cv, y_cv, cv=kfold,scoring="accuracy")
        names.append(name)
        results.append(cv_result)
        print("{} modelinin çapraz doğrulaması yapıldı, başarı oranı:{:0.2f}".format(name, cv_result.mean()))

def cross_validate(dataset, target_name):
    X=dataset.drop(target_name, axis=1)
    y=dataset[target_name]

    X=StandardScaler().fit_transform(X)
    
    #labelEncoder=LabelEncoder()
    #y=labelEncoder.fit_transform(y)

    cross_validation_scores_for_various_ml_models(X, y)

<a class="anchor" id="5.3."></a>**5.3. Çapraz Doğrulama Sonuçları**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
cross_validate(dataset=dataset, target_name="target")

<a class="anchor" id="5.4."></a>**5.4. Yardımcı Fonksiyon: SVM_GridSearch()**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
def SVM_GridSearch(dataset, target_name):
    X=dataset.drop(target_name, axis=1)
    X=StandardScaler().fit_transform(X)
    
    y=dataset[target_name]
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)
    best_score=0
    gammas=[0.001, 0.01, 0.1, 1, 10, 100]
    Cs=[0.001, 0.01, 0.1, 1, 10, 100]
    kernels=['rbf', 'linear']
    
    for gamma in gammas:
        for C in Cs:
            for kernel in kernels:
                svm=SVC(kernel=kernel,gamma=gamma, C=C)
                svm.fit(X_train, y_train)


                score=svm.score(X_test, y_test)

                if score>best_score:
                    y_pred=svm.predict(X_test)
                    best_score=score
                    best_params={'kernel':kernel, 'C':C, 'gamma':gamma}
        
    print("best score:",best_score)
    print("best params:",best_params)
    print("classification reports:\n",classification_report(y_test, y_pred))

<a class="anchor" id="5.5."></a>**5.5. En İyi Sonuç Veren SVC Parametrelerinin Elde Edilmesi**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
# bu işlem 10 dakkika kadar sürmektedir
#SVM_GridSearch(dataset=dataset, target_name='target')
# Elde edilen en iyi parametreler;
# best params: {'kernel': 'linear', 'C': 0.01, 'gamma': 0.001}

<a class="anchor" id="5.6."></a>**5.6. En İyi SVC Parametrelerine Göre Elde Edilen Sonuçlar**<=======> [İçindekiler Menüsüne Git](#0.)

In [ ]:
def SVC_use_best_params(X, y):
    X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

    svm=SVC(kernel='linear',gamma= 0.001, C=0.01)
    svm.fit(X_train, y_train)
    
    score=svm.score(X_test, y_test)
    y_pred=svm.predict(X_test)
    
    print("Başarı oranı:{:.2f}".format(score))
    print("classification reports:\n",classification_report(y_test, y_pred))

In [ ]:
X=dataset.drop('target', axis=1)
X=StandardScaler().fit_transform(X)
    
y=dataset['target']

SVC_use_best_params(X, y)


<a class="anchor" id="5.7."></a>**5.7. ROC ve AUC**<=======> [İçindekiler Menüsüne Git](#0.)

ROCAUC (Receiver Operating Characteristic/Area Under the Curve) grafiği, sınıflandırıcının duyarlılığı ve özgüllüğü arasındaki farkın görünmesini sağlar.

In [ ]:
from yellowbrick.classifier import ROCAUC

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape,X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
fig, ax=plt.subplots(1,1,figsize=(12,8))

best_clf=SVC(kernel='linear',gamma= 0.001, C=0.01)

roc_auc=ROCAUC(best_clf, ax=ax)

roc_auc=ROCAUC(best_clf, ax=ax)
roc_auc.fit(X_train, y_train)

"""
Bu kısısmda sebebini anlayamadığım bir hata veriyor. Bu nedenle pasif hale getirdim
"""
#roc_auc.score(X_test, y_test)

roc_auc.poof()

[İçindekiler Menüsüne Git](#0.)

# <a class="anchor" id="6."></a>**6. Gözetimisiz Öğrenme** 
Yukarıda yaptığımız sınıflandırma işlemi, gözetimli makine öğrenmesi yaklaşımıdır. Gözetimli öğrenmede veri seti 'veri' ve 'çıktı' olmak üzere iki ana bölüme ayrılır. Veri, veri setindeki örneğin değerlerini tutarken 'çıktı' örneğin ait olduğu sınıfı(sınıflandırma için) veya hedef değeri(regresyon için) tutar. Gözetimsiz öğrenmede veri seti sadece veri bölümünden oluşur.

Gözetimsiz öğrenme genelde ikiye ayrılır: verinin dönüştürülmesi ve kümeleme. Bu çalışmada gözetimsiz öğrenme kullanılarak verinin dönüştürülmesi gerçekleştirilecektir. Gözetimsiz dönüştürme yöntemleri verinin bilgisayarlar ve insanlar tarafından daha kolay yorumlanmasını sağlar.

En yaygın gözetimli dönüşüm uygulamaları veri boyutunun azaltılmasıdır. Boyut azaltma işleminde çok boyutlu veri daha az boyutla temsil edilir. 


<a class="anchor" id="6.1."></a>**6.1. Temel Bileşen Analizi**<======>[İçindekiler Menüsüne Git](#0.)

Temel Bileşen Analizi(Principle Component Analiysis(PCA)) verinin daha az boyutta temsil edilmesini sağlayan bir yöntemdir. Bu yönteme göre veri yeni bileşenlere dönüştürülür ve en önemli bileşenler secilerek verinin boyutu azaltılır. 

In [ ]:
import mglearn

In [ ]:
mglearn.plots.plot_pca_illustration()

Yukarıdaki çizim, sentetik iki boyutlu veri kümesi üzerinde basit bir örnek göstermektedir. İlk çizim, noktaları ayırt etmek için renklendirilmiş orijinal veri noktalarını gösterir. Algoritma, önce "Bileşen 1" olarak etiketlenen maksimum varyansın yönünü bularak ilerler. Bu, verilerin çoğunu içeren ya da başka bir deyişle, özelliklerin birbiriyle en fazla ilişkili olduğu yönü ifade eder.

Daha sonra algoritma, ortogonal (sağ açıda) iken ilk yöne en fazla bilgiyi içeren yönü bulur. İki boyutta, doğru bir açıda olan tek bir olası yönelim vardır, fakat yüksek boyutlu boşluklarda (sonsuz) birçok ortogonal yön olacaktır.

**Kaynak:** Introduction to Machine Learning with Python_ A Guide for Data Scientists-O'Reilly Media (2016)

<a class="anchor" id="6.2."></a>**6.2. PCA Dönüşümünün Gerçekleştirilmesi**<=====>[İçindekiler Menüsüne Git](#0.)

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
pca.fit(X)

X_pca=pca.transform(X)
print("PCA dönüşümü yapıldı")

In [ ]:
print("Verinin orijinal boyutu:{}".format(X.shape))
print("Dönüşümden sonraki boyutu:{}".format(X_pca.shape))

<a class="anchor" id="6.3."></a>**6.3. PCA Değerlerinin Görselleştirilmesi**<=====>[İçindekiler Menüsüne Git](#0.)

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(X_pca[:,0],X_pca[:,1], c=y,s=60)
plt.gca().set_aspect('equal')
plt.xlabel("Birinci Temel Bileşen")
plt.ylabel("İkinci Temel Bileşen")
plt.show()

<a class="anchor" id="6.4."></a>**6.4. Sadece İki PCA Bileşeniyle Elde Edilen Sonuçlar**<=====>[İçindekiler Menüsüne Git](#0.)

In [ ]:
SVC_use_best_params(X_pca, y)

<a class="anchor" id="6.5."></a>**6.5. Optimum PCA Bileşen Sayısının Belirlenmesi** <=====>[İçindekiler MenüsüneGit](#0.)

In [ ]:
pca=PCA()
pca.fit(X)
plt.figure(1, figsize=(12,8))
plt.clf()
plt.axes([.2, .2, .7, .7])
plt.plot(pca.explained_variance_, linewidth=2)
plt.axis('tight')
plt.xlabel('Bileşen Sayısı')
plt.ylabel('Orijinal Veriyle Temsil Farkı')
plt.show()

<a class="anchor" id="6.6."></a>**6.6. Optimum PCA Bileşen Sayısına Göre Sonuçlar** <=====>[İçindekiler Menüsüne Git](#0.)

Yukardaki şekilde 200 ve daha fazla PCA bileşeninin veriyi aynı ölçüde temsil ettiği görülebiliyor. Şimdi 200 PCA bileşeni kullanarak sınıflandırma işlemi yapalım. 

In [ ]:
pca=PCA(n_components=200)
pca.fit(X)
X_pca=pca.transform(X)
print("PCA dönüşümü gerçekleştirildi")

In [ ]:
print("Verinin orijinal boyutu:{}".format(X.shape))
print("Dönüşümden sonraki boyutu:{}".format(X_pca.shape))

In [ ]:
SVC_use_best_params(X_pca, y)

<a class="anchor" id="7."></a>**7. Makine Öğrenmesi İş Akışının Otomatikleştirilmesi**

Bu kısma Pipeline anlatımı ve kodlaması gelecek...

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pca=PCA(n_components=200)
clf=SVC(kernel='linear',gamma= 0.001, C=0.01)
pipeline=Pipeline([("pca",pca), ("svm", clf)])
print("Pipeline  oluşturuldu...")

In [ ]:
pipeline.fit(X_train, y_train)
print("Pipeline eğitildi...")

In [ ]:
y_pred=pipeline.predict(X_test)
print("Pipeline ile test verileri üzerinde tahmin yapıldı...")

In [ ]:
print("Başarı oranı:{:.2f}".format(accuracy_score(y_test, y_pred)))

<a class="anchor" id="8."></a>**8. Makine Öğrenmesi Modelinin Kaydedilmesi ve Yüklenmesi**

Bu kısma Serialization anlatımı ve kodlaması gelecek...